# Tidy Data

1. Attendance Data

- Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

- You should end up with something like this:
name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64



In [28]:
import pandas as pd
import numpy as np
import env
import acquire
import prepare
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [29]:
def get_attendance_data():
    if os.path.isfile('attendance.csv'):
        return pd.read_csv('attendance.csv')
    else:
        url = env.get_connection('tidy_data')
        query = '''
                SELECT *
                FROM attendance
                '''
        attendance = pd.read_sql(query, url)
        # attendance = attendance.drop(columns = 'Unnamed: 01')
        attendance.to_csv('attendance.csv')
        return attendance

In [30]:
attendance = get_attendance_data()
attendance.head()

attendance.rename(columns = {'Unnamed: 0':'name'}, inplace= True)
# attendance



In [31]:
attendance.info() #need to drop Unnamed: 0.1 column
attendance = attendance.drop(columns= 'Unnamed: 0.1')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  4 non-null      int64 
 1   name          4 non-null      object
 2   2018-01-01    4 non-null      object
 3   2018-01-02    4 non-null      object
 4   2018-01-03    4 non-null      object
 5   2018-01-04    4 non-null      object
 6   2018-01-05    4 non-null      object
 7   2018-01-06    4 non-null      object
 8   2018-01-07    4 non-null      object
 9   2018-01-08    4 non-null      object
dtypes: int64(1), object(9)
memory usage: 448.0+ bytes


In [32]:
attendance
#next use .melt then .groupby

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [33]:
attendance = attendance.melt(id_vars=['name'], var_name='days', value_name='attendance')

In [34]:
# map  values to the letters in attendance column

attendance['attendance'] = attendance['attendance'].map({'A':0,
                                                         'H': 0.5,
                                                         'T': 0.9,
                                                         'P':1})

In [35]:
attendance.head()

,name,days,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [36]:
attendance = attendance.groupby('name')['attendance'].mean()

In [37]:
attendance

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance, dtype: float64


2. Coffee Levels

- Read the coffee_levels table.
- Transform the data so that each carafe is in it's own column.
- Is this the best shape for the data?



In [38]:
def get_coffee_data():
    if os.path.isfile('coffee_levels.csv'):
        return pd.read_csv('coffee_levels.csv')
    else:
        url = env.get_connection('tidy_data')
        query = '''
                SELECT *
                FROM coffee_levels;
                '''
        coffee_levels = pd.read_sql(query, url)
        coffee_levels.to_csv('coffee_levels.csv')
        return coffee_levels

In [39]:
coffee_db = get_coffee_data()

In [40]:
coffee_db.info()
coffee_db.head()
coffee_db = coffee_db.drop(columns = 'Unnamed: 0')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     30 non-null     int64  
 1   hour           30 non-null     int64  
 2   coffee_carafe  30 non-null     object 
 3   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ KB


In [41]:
coffee_db.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [42]:
encoded_vars = pd.get_dummies(coffee_db[['coffee_carafe']])


In [43]:
coffee_encoded = pd.concat([coffee_db, encoded_vars], axis=1)
coffee_encoded = coffee_encoded.drop(columns='coffee_carafe')
coffee_encoded

,hour,coffee_amount,coffee_carafe_x,coffee_carafe_y,coffee_carafe_z
0,8,0.816164,1,0,0
1,9,0.451018,1,0,0
2,10,0.843279,1,0,0
3,11,0.335533,1,0,0
4,12,0.898291,1,0,0
5,13,0.310711,1,0,0
6,14,0.507288,1,0,0
7,15,0.215043,1,0,0
8,16,0.183891,1,0,0
9,17,0.391560,1,0,0


In [45]:
coffee_encoded.shape
coffee_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hour             30 non-null     int64  
 1   coffee_amount    30 non-null     float64
 2   coffee_carafe_x  30 non-null     uint8  
 3   coffee_carafe_y  30 non-null     uint8  
 4   coffee_carafe_z  30 non-null     uint8  
dtypes: float64(1), int64(1), uint8(3)
memory usage: 698.0 bytes



3. Cake Recipes

- Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- Tidy the data as necessary.
- Which recipe, on average, is the best? recipe b
- Which oven temperature, on average, produces the best results? 275
- Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees



4. Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.